In [29]:
import sys
sys.path.append('../')

import os
import time
import datetime
import tqdm
import numpy as np
from logger import logger
import pandas as pd
from utils.misc_utils import RMSE, MAE, masked_mape_np

from dnn.tai_test import TAI
from utils.tools import addResult,plotResult
from ycimpute.utils import evaluate
from utils.base_tools import modifier
from utils.dataAnalysis import analysisMeanMain,json2Excel
from baseline.SOTABaselineMICE import  imputeMethodMICE as MICE
from utils.tools import saveJson
from utils.read_file import readAllTypeFile
from utils.handle_missingdata import gene_missingdata, gene_missingdata_taxa_bias, gene_missingdata_chara_bias, \
    gene_missingdata_block_bias

# 参数修改测试 epochs=500 theta=int(len(missData[0]) / 3) iterations=1000

In [30]:
def imputeMethod(result,loss,firstImputedMethod,autoMethod,originData,missData,missRate,missPattern,dataType='continuous',firstImputedData="None"):
    imputationMethod =  "{}_{}_{}".format(firstImputedMethod, loss, autoMethod)
    try:
        imputedData, firstImputedData = TAI(first_imputation_method=firstImputedMethod,
                                            firstImputedData=firstImputedData,
                                               batch_size=len(missData),
                                               epochs=500,
                                               theta=int(len(missData[0]) / 3),
                                               iterations=1000,
                                               Autoencoder_method=autoMethod,
                                               loss=loss,
                                               use_cuda=False
                                               ).complete(missData)
        
        if dataType!='continuous':
            mark = [temp[0] for temp in pd.DataFrame(np.unique(missData)).dropna(axis=0).values]
            imputedData = modifier(imputedData, mark)

        result = addResult(result, missRate, missPattern, imputationMethod,
                           evaluate.RMSE(originData, imputedData),
                           MAE(originData, imputedData),
                           masked_mape_np(originData, imputedData))

        return result, imputedData
    except Exception as e:
        print(e)
        result = addResult(result, missRate, missPattern, imputationMethod,
                           np.inf,
                           np.inf,
                           np.inf)

        return result,firstImputedData

In [31]:
def mainWork(path,savePath):
    pbar = tqdm.tqdm(os.listdir(path), desc='dirs')
    for file in pbar:
        pbar.set_description("Processing %s" % file)
        if file.endswith('xlsx') or file.endswith('csv'):
            originData = readAllTypeFile(os.path.join(path, file))
            for missPattern in ['normal']:
                result = {}
                for missRate in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
                    if missPattern == 'normal':
                        missData = gene_missingdata(rate=missRate, data=originData)
                    elif missPattern == 'taxa':
                        missData = gene_missingdata_taxa_bias(rate=missRate, data=originData)
                    elif missPattern == 'chara':
                        missData = gene_missingdata_chara_bias(rate=missRate, data=originData)
                    elif missPattern == 'block':
                        missData = gene_missingdata_block_bias(rate=missRate, data=originData)
                    else:
                        raise Exception("缺失模式错误，请在'normal','taxa','chara','block'中选择对应模式")

                    result, MICEImputedData = MICE(result, originData, missData, missRate, missPattern)
                    for firstImputedMethod in ['mice']:
                        if firstImputedMethod=='ii':
                            firstImputedData=IImputedData
                        elif firstImputedMethod=='mice':
                            firstImputedData  = MICEImputedData
                        elif firstImputedMethod =='knn':
                            firstImputedData=KNNImputedData
                        for loss in ['MSELoss']:
                            for autoMethod in ['StockedResAutoencoder']:
                                start=time.time()
                                result,_=imputeMethod(result=result,firstImputedMethod=firstImputedMethod,
                                                    firstImputedData=firstImputedData,
                                                    loss=loss,autoMethod=autoMethod,
                                                    originData=originData,missData=missData,
                                                    missRate=missRate,missPattern=missPattern)
                                logger.info("{}-{}-{}训练耗时:{}".format(firstImputedMethod,loss,autoMethod,time.time() - start))
                saveJson(result, os.path.join(savePath,"{}_{}_{}_{}.json".format("allMethod", missPattern,file, datetime.datetime.now().strftime('%Y%m%d-%H%M%S'))))

In [32]:
path = r'../public_data'
savePath=r'C:\Users\pro\Desktop\实验二自编码器建树\UCI数据集测试\自编码器方法参数测试\E500I1000T3'
for _ in range(5):
    mainWork(path,savePath)


Processing 10_Cancer.xlsx:   0%|                                                                                                              | 0/14 [00:00<?, ?it/s]2021-01-27 14:52:02.816 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时5.647996664047241


Train Epoch: 0 [0/567 (0%)]	Loss: 0.010091
Train Epoch: 1 [0/567 (0%)]	Loss: 0.006712
Train Epoch: 2 [0/567 (0%)]	Loss: 0.004463
Train Epoch: 3 [0/567 (0%)]	Loss: 0.003088
Train Epoch: 4 [0/567 (0%)]	Loss: 0.002320
Train Epoch: 5 [0/567 (0%)]	Loss: 0.001862
Train Epoch: 6 [0/567 (0%)]	Loss: 0.001511
Train Epoch: 7 [0/567 (0%)]	Loss: 0.001201
Train Epoch: 8 [0/567 (0%)]	Loss: 0.000938
Train Epoch: 9 [0/567 (0%)]	Loss: 0.000730
Train Epoch: 10 [0/567 (0%)]	Loss: 0.000563
Train Epoch: 11 [0/567 (0%)]	Loss: 0.000431
Train Epoch: 12 [0/567 (0%)]	Loss: 0.000337
Train Epoch: 13 [0/567 (0%)]	Loss: 0.000287
Train Epoch: 14 [0/567 (0%)]	Loss: 0.000274
Train Epoch: 15 [0/567 (0%)]	Loss: 0.000285
Train Epoch: 16 [0/567 (0%)]	Loss: 0.000302
Train Epoch: 17 [0/567 (0%)]	Loss: 0.000310
Train Epoch: 18 [0/567 (0%)]	Loss: 0.000299
Train Epoch: 19 [0/567 (0%)]	Loss: 0.000272
Train Epoch: 20 [0/567 (0%)]	Loss: 0.000234
Train Epoch: 21 [0/567 (0%)]	Loss: 0.000196
Train Epoch: 22 [0/567 (0%)]	Loss: 0.00016

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-27 14:52:05.081 | INFO     | __main__:mainWork:37 - mice-MSELoss-StockedResAutoencoder训练耗时:2.2639739513397217
2021-01-27 14:52:11.098 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时6.012025833129883


Train Epoch: 0 [0/567 (0%)]	Loss: 0.008112
Train Epoch: 1 [0/567 (0%)]	Loss: 0.005703
Train Epoch: 2 [0/567 (0%)]	Loss: 0.004074
Train Epoch: 3 [0/567 (0%)]	Loss: 0.003033
Train Epoch: 4 [0/567 (0%)]	Loss: 0.002404
Train Epoch: 5 [0/567 (0%)]	Loss: 0.001999
Train Epoch: 6 [0/567 (0%)]	Loss: 0.001658
Train Epoch: 7 [0/567 (0%)]	Loss: 0.001306
Train Epoch: 8 [0/567 (0%)]	Loss: 0.000959
Train Epoch: 9 [0/567 (0%)]	Loss: 0.000668
Train Epoch: 10 [0/567 (0%)]	Loss: 0.000464
Train Epoch: 11 [0/567 (0%)]	Loss: 0.000342
Train Epoch: 12 [0/567 (0%)]	Loss: 0.000280
Train Epoch: 13 [0/567 (0%)]	Loss: 0.000254
Train Epoch: 14 [0/567 (0%)]	Loss: 0.000250
Train Epoch: 15 [0/567 (0%)]	Loss: 0.000257
Train Epoch: 16 [0/567 (0%)]	Loss: 0.000263
Train Epoch: 17 [0/567 (0%)]	Loss: 0.000257
Train Epoch: 18 [0/567 (0%)]	Loss: 0.000238
Train Epoch: 19 [0/567 (0%)]	Loss: 0.000210
Train Epoch: 20 [0/567 (0%)]	Loss: 0.000179
Train Epoch: 21 [0/567 (0%)]	Loss: 0.000151
Train Epoch: 22 [0/567 (0%)]	Loss: 0.00012

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-27 14:52:13.423 | INFO     | __main__:mainWork:37 - mice-MSELoss-StockedResAutoencoder训练耗时:2.3249740600585938
2021-01-27 14:52:20.009 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时6.579030752182007


...

Train Epoch: 0 [0/208 (0%)]	Loss: 0.064669
Train Epoch: 1 [0/208 (0%)]	Loss: 0.061755
Train Epoch: 2 [0/208 (0%)]	Loss: 0.058923
Train Epoch: 3 [0/208 (0%)]	Loss: 0.056178
Train Epoch: 4 [0/208 (0%)]	Loss: 0.053519
Train Epoch: 5 [0/208 (0%)]	Loss: 0.050946
Train Epoch: 6 [0/208 (0%)]	Loss: 0.048457
Train Epoch: 7 [0/208 (0%)]	Loss: 0.046052
Train Epoch: 8 [0/208 (0%)]	Loss: 0.043729
Train Epoch: 9 [0/208 (0%)]	Loss: 0.041489
Train Epoch: 10 [0/208 (0%)]	Loss: 0.039332
Train Epoch: 11 [0/208 (0%)]	Loss: 0.037255
Train Epoch: 12 [0/208 (0%)]	Loss: 0.035258
Train Epoch: 13 [0/208 (0%)]	Loss: 0.033341
Train Epoch: 14 [0/208 (0%)]	Loss: 0.031500
Train Epoch: 15 [0/208 (0%)]	Loss: 0.029735
Train Epoch: 16 [0/208 (0%)]	Loss: 0.028044
Train Epoch: 17 [0/208 (0%)]	Loss: 0.026426
Train Epoch: 18 [0/208 (0%)]	Loss: 0.024878
Train Epoch: 19 [0/208 (0%)]	Loss: 0.023398
Train Epoch: 20 [0/208 (0%)]	Loss: 0.021985
Train Epoch: 21 [0/208 (0%)]	Loss: 0.020637
Train Epoch: 22 [0/208 (0%)]	Loss: 0.01935

Train Epoch: 239 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 240 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 241 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 242 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 243 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 244 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 245 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 246 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 247 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 248 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 249 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 250 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 251 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 252 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 253 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 254 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 255 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 256 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 257 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 258 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 259 [0/208 (0%)]	Loss: 0.000001
Train Epoch: 260 [0/208 (0%)]	Loss: 0.000001
Train Epoc

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-27 17:05:49.063 | INFO     | __main__:mainWork:37 - mice-MSELoss-StockedResAutoencoder训练耗时:1.8359999656677246
2021-01-27 17:05:49.739 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.6739990711212158


Train Epoch: 0 [0/208 (0%)]	Loss: 0.061427
Train Epoch: 1 [0/208 (0%)]	Loss: 0.059248
Train Epoch: 2 [0/208 (0%)]	Loss: 0.057133
Train Epoch: 3 [0/208 (0%)]	Loss: 0.055081
Train Epoch: 4 [0/208 (0%)]	Loss: 0.053089
Train Epoch: 5 [0/208 (0%)]	Loss: 0.051156
Train Epoch: 6 [0/208 (0%)]	Loss: 0.049281
Train Epoch: 7 [0/208 (0%)]	Loss: 0.047463
Train Epoch: 8 [0/208 (0%)]	Loss: 0.045702
Train Epoch: 9 [0/208 (0%)]	Loss: 0.043994
Train Epoch: 10 [0/208 (0%)]	Loss: 0.042338
Train Epoch: 11 [0/208 (0%)]	Loss: 0.040728
Train Epoch: 12 [0/208 (0%)]	Loss: 0.039163
Train Epoch: 13 [0/208 (0%)]	Loss: 0.037637
Train Epoch: 14 [0/208 (0%)]	Loss: 0.036146
Train Epoch: 15 [0/208 (0%)]	Loss: 0.034688
Train Epoch: 16 [0/208 (0%)]	Loss: 0.033259
Train Epoch: 17 [0/208 (0%)]	Loss: 0.031856
Train Epoch: 18 [0/208 (0%)]	Loss: 0.030478
Train Epoch: 19 [0/208 (0%)]	Loss: 0.029122
Train Epoch: 20 [0/208 (0%)]	Loss: 0.027787
Train Epoch: 21 [0/208 (0%)]	Loss: 0.026472
Train Epoch: 22 [0/208 (0%)]	Loss: 0.02517

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-27 17:05:51.209 | INFO     | __main__:mainWork:37 - mice-MSELoss-StockedResAutoencoder训练耗时:1.4690003395080566
Processing tmp: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████| 14/14 [26:53<00:00, 115.29s/it]


In [33]:
from utils.dataAnalysis import analysisMeanMain,analysisPlot
analysisMeanMain(savePath, savePath)

+-----------------------------------+--------+------------------------------------+----------+-----------------+-------------+-------------+
|              dataSet              | Method |              pattern               | missRate |       RMSE      |     MAE     |     MAPE    |
+-----------------------------------+--------+------------------------------------+----------+-----------------+-------------+-------------+
|  allMethod_normal_10_Cancer.xlsx  | normal |                MICE                |   0.05   |   4.040+1.363   | 0.114+0.022 | 0.413+0.028 |
|  allMethod_normal_10_Cancer.xlsx  | normal | mice_MSELoss_StockedResAutoencoder |   0.05   |   4.040+1.364   | 0.113+0.022 | 0.414+0.028 |
| allMethod_normal_12_Banknote.xlsx | normal |                MICE                |   0.05   |   0.334+0.034   | 0.048+0.003 | 9.789+5.163 |
| allMethod_normal_12_Banknote.xlsx | normal | mice_MSELoss_StockedResAutoencoder |   0.05   |   0.334+0.034   | 0.048+0.003 | 9.801+5.153 |
|  allMethod_

+-----------------------------------+--------+------------------------------------+----------+----------------+--------------+-----------------------+
|              dataSet              | Method |              pattern               | missRate |      RMSE      |     MAE      |          MAPE         |
+-----------------------------------+--------+------------------------------------+----------+----------------+--------------+-----------------------+
|  allMethod_normal_10_Cancer.xlsx  | normal |                MICE                |   0.50   |  22.732+4.030  | 2.378+0.319  |      8.490+0.294      |
|  allMethod_normal_10_Cancer.xlsx  | normal | mice_MSELoss_StockedResAutoencoder |   0.50   |  22.732+4.030  | 2.378+0.319  |      8.491+0.293      |
| allMethod_normal_12_Banknote.xlsx | normal |                MICE                |   0.50   |  2.002+0.027   | 0.918+0.015  |   2796.071+3455.718   |
| allMethod_normal_12_Banknote.xlsx | normal | mice_MSELoss_StockedResAutoencoder |   0.50   |